# Minst

* 하나의 히든 레이어를 도입한다.
* input layer는 28*28개의 노드
* hidden layer의 노드의 수는 자유롭게 설정해도 좋다.
* output은 10개(0~9)로 설정한다.

### test data를 예측하여 정확도를 최대가 되도록 학습하여라.

* 제출 시 test data의 정확도를 출력하여 제출하세요.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision

train_data = torchvision.datasets.MNIST(
    root = 'data',
    train = True,                         
    transform = torchvision.transforms.ToTensor(), 
    download = True,            
)
test_data = torchvision.datasets.MNIST(
    root = 'data', 
    train = False, 
    transform = torchvision.transforms.ToTensor()
)

train_data

test_data # 제출시 이 test 데이터에 대한 정확도를 출력하세요.

Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [2]:
X = torch.flatten(train_data.data, start_dim=1).float()
y = train_data.targets
X=X/255 

In [3]:
din,dh,dout = 784, [144,64], 10

model = torch.nn.Sequential(
    torch.nn.Linear(din, dh[0]),
    torch.nn.ReLU(),
    torch.nn.Linear(dh[0], dh[1]),
    torch.nn.ReLU(),
    torch.nn.Linear(dh[1], dout),
    torch.nn.Softmax(dim=1)
)

learning_rate = 0.001
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

In [4]:
# learning
nums_epoch = 1000
def train(model, criterion, optimizer, nums_epoch, X,y):
    for epoch in range(nums_epoch):
        y_pred = model(X)
        loss = criterion(y_pred,y)
        optimizer.zero_grad() # zero_grad(): 미분값 초기화
        loss.backward() # backpropagation
        optimizer.step() # update weights and biases
        
        if(epoch%100==0):
            prediction = y_pred.max(1)[1] # max indices
            corrects = (prediction == y)
            accuracy = corrects.sum().float() / float( y.size(0) )
            print(epoch, "loss=", loss.item(), "accu=", accuracy.item())

In [6]:
train(model,criterion, optimizer,nums_epoch, X, y);

0 loss= 2.30273175239563 accu= 0.10048333555459976
100 loss= 1.6312426328659058 accu= 0.8428500294685364
200 loss= 1.6040241718292236 accu= 0.8628166913986206
300 loss= 1.5924166440963745 accu= 0.8727333545684814
400 loss= 1.5172631740570068 accu= 0.9504166841506958
500 loss= 1.5020148754119873 accu= 0.9644500017166138
600 loss= 1.493234395980835 accu= 0.971916675567627
700 loss= 1.4874448776245117 accu= 0.9768999814987183
800 loss= 1.4830926656723022 accu= 0.980566680431366
900 loss= 1.480494499206543 accu= 0.982366681098938


In [7]:
# training
nums_epoch = 1000
def train_val(model, criterion, optimizer, nums_epoch, X, y,X_test, y_test):
    for epoch in range(nums_epoch):
        y_pred = model(X)
        loss = criterion(y_pred,y)
        
        optimizer.zero_grad() # zero_grad(): 미분값 초기화
        loss.backward() # backpropagation
        optimizer.step() # update weights and biases
        
        if(epoch%100==0):
            y_pred = model(X)
            prediction = y_pred.max(1)[1] # max indices
            corrects = (prediction == y)
            accuracy_train = corrects.sum().float() / float( y.size(0) )
            
            y_test_pred = model(X_test)
            test_prediction = y_test_pred.max(1)[1] # max indices
            test_corrects = (test_prediction == y_test)
            accuracy_test = test_corrects.sum().float() / float( y_test_pred.size(0) )
            print(epoch, "train=", accuracy_train.item(), "test=", accuracy_test.item())

In [8]:
X_test=torch.flatten(test_data.data,start_dim=1).float()
y_test=test_data.targets
X_test=X_test/255 

In [9]:
train_val(model, criterion, optimizer, nums_epoch, X, y , X_test, y_test)

0 train= 0.9835000038146973 test= 0.9627000093460083
100 train= 0.9844333529472351 test= 0.9632999897003174
200 train= 0.9850833415985107 test= 0.9639000296592712
300 train= 0.9856666922569275 test= 0.9642999768257141
400 train= 0.9860666394233704 test= 0.9646000266075134
500 train= 0.986299991607666 test= 0.964900016784668
600 train= 0.9865833520889282 test= 0.9652000069618225
700 train= 0.9869999885559082 test= 0.9646999835968018
800 train= 0.9873499870300293 test= 0.9661999940872192
900 train= 0.9876333475112915 test= 0.9656999707221985
